In [30]:
print("let's create company's brochure")

let's create company's brochure


In [3]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [5]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
# print(api_key)

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [6]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
res = Website("https://www.deeplearning.ai/courses/machine-learning-specialization/")
res.links

['https://bit.ly/43WQcwy',
 '/',
 '/courses/',
 '/the-batch/',
 '/the-batch/',
 '/the-batch/tag/letters/',
 '/the-batch/tag/data-points/',
 '/the-batch/tag/research/',
 '/blog/',
 '/community/',
 'https://community.deeplearning.ai/',
 '/events/',
 '/ambassador/',
 '/blog/category/ambassador-spotlight/',
 '/resources/',
 '/about/',
 '/about/',
 '/careers/',
 '/contact/',
 'https://bit.ly/3RB9T8a',
 '/courses/',
 '/courses/machine-learning-specialization/',
 '/courses/machine-learning-specialization/#overview',
 '/courses/machine-learning-specialization/#instructors',
 '/courses/machine-learning-specialization/#syllabus',
 '/courses/machine-learning-specialization/#course-slides',
 '/courses/machine-learning-specialization/#faqs',
 'https://www.coursera.org/specializations/machine-learning-introduction?utm_source=deeplearning-ai&utm_medium=institutions&utm_campaign=WebsiteCourses-MLS',
 'https://bit.ly/3aTxAFT',
 'https://www.deeplearning.ai/program/deep-learning-specialization/',
 'http

In [9]:
# get only relevant links

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [10]:
print(link_system_prompt)


You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [11]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [12]:
print(get_links_user_prompt(res))


Here is the list of links on the website of https://www.deeplearning.ai/courses/machine-learning-specialization/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://bit.ly/43WQcwy
/
/courses/
/the-batch/
/the-batch/
/the-batch/tag/letters/
/the-batch/tag/data-points/
/the-batch/tag/research/
/blog/
/community/
https://community.deeplearning.ai/
/events/
/ambassador/
/blog/category/ambassador-spotlight/
/resources/
/about/
/about/
/careers/
/contact/
https://bit.ly/3RB9T8a
/courses/
/courses/machine-learning-specialization/
/courses/machine-learning-specialization/#overview
/courses/machine-learning-specialization/#instructors
/courses/machine-learning-specialization/#syllabus
/courses/machine-learning-specialization/#course-slides
/courses/machine-learning-specialization/#faqs
https://www.coursera.org/sp

In [13]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [14]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/mistralai/Magistral-Small-2506',
 '/nanonets/Nanonets-OCR-s',
 '/echo840/MonkeyOCR',
 '/tencent/Hunyuan3D-2.1',
 '/openbmb/MiniCPM4-8B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/ResembleAI/Chatterbox',
 '/spaces/aisheets/sheets',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/Agents-MCP-Hackathon/AI-Marketing-Content-Creator',
 '/spaces',
 '/datasets/nvidia/Nemotron-Personas',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/open-thoughts/OpenThoughts3-1.2M',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/a-m-team/AM-DeepSeek-R1-0528-Distilled',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',

In [15]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

In [16]:
get_links("https://www.deeplearning.ai/courses/machine-learning-specialization/")

{'links': [{'type': 'about page', 'url': 'https://www.deeplearning.ai/about/'},
  {'type': 'careers page', 'url': 'https://www.deeplearning.ai/careers/'}]}

In [17]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [19]:
# print(get_all_details("https://huggingface.co"))


In [20]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [21]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [22]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")


Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmistralai/Magistral-Small-2506\nUpdated\n4 days ago\n•\n11.6k\n•\n408\nnanonets/Nanonets-OCR-s\nUpdated\nabout 16 hours ago\n•\n4.25k\n•\n242\necho840/MonkeyOCR\nUpdated\n3 days ago\n•\n4\n•\n217\ntencent/Hunyuan3D-2.1\nUpdated\nabout 6 hours ago\n•\n168\nopenbmb/MiniCPM4-8B\nUpdated\n2 days ago\n•\n5.81k\n•\n240\nBrowse 1M+ models\nSpaces\nRunning\n8.06k\n8.06k\nDeepSite\n🐳\nGenerate any 

In [23]:
get_brochure_user_prompt("DeepLearning.AI", "https://www.deeplearning.ai/courses/machine-learning-specialization/")


Found links: {'links': [{'type': 'about page', 'url': 'https://www.deeplearning.ai/about/'}, {'type': 'careers page', 'url': 'https://www.deeplearning.ai/careers/'}, {'type': 'community page', 'url': 'https://community.deeplearning.ai/'}]}


"You are looking at a company called: DeepLearning.AI\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nMachine Learning Specialization - DeepLearning.AI\nWebpage Contents:\n✨ New course! Enroll in\nOrchestrating Workflows for GenAI Applications\nExplore Courses\nAI Newsletter\nThe Batch\nAndrew's Letter\nData Points\nML Research\nBlog\nCommunity\nForum\nEvents\nAmbassadors\nAmbassador Spotlight\nResources\nCompany\nAbout\nCareers\nContact\nStart Learning\nCourses\nMachine Learning Specialization\nOverview\nInstructors\nSyllabus\nCourse Slides\nFAQs\nEnroll Now\nStarting Your AI Career Has Never Been Easier\n#BreakIntoAI\nwith the new Machine Learning Specialization, an updated foundational program for beginners created by Andrew Ng.\nEnroll Now\nPresenting the new\nMachine Learning Specialization\nAbout the original course\n2012\nYear launched\nRated 4.9 out of

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")


Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

---

## Welcome to Hugging Face

**The AI community building the future.** Hugging Face serves as a collaboration hub for the machine learning community, where users can explore, create, and advance models, datasets, and applications within an open environment. 

---

### What We Offer

- **Models**: Browse and collaborate on over 1 million machine learning models.
- **Datasets**: Access a rich repository of over 250k datasets to enhance your projects and research.
- **Spaces**: Create and run applications effortlessly, empowering developers and creators alike.
- **Enterprise Solutions**: Tailored services including enterprise-grade security, access control, and dedicated support, catering to teams of all sizes.
  
Explore trending models and applications within our vibrant community, including popular AI-powered tools for marketing content generation, OCR, and more.

---

### Our Community

At Hugging Face, we believe in the power of collaboration. More than 50,000 organizations, including industry leaders like Google, Microsoft, and Amazon, use our platform. Our vibrant community continues to evolve, fueling innovation in AI and machine learning. Whether you’re a simple user or representing a large enterprise, you’ll find invaluable resources and support.

---

### Company Culture

- **Open-Source Impact**: We are dedicated to democratizing machine learning. Contribution from community members is not only welcomed but encouraged as we build the future of AI together.
- **Innovation Driven**: Our culture is steeped in creativity and experimentation. We continuously strive to push the boundaries of what’s possible with machine learning.
- **Inclusivity**: A collaborative environment where everyone can share ideas and insights freely, fostering cross-disciplinary teamwork and knowledge-sharing.

---

### Careers at Hugging Face

Join a passionate team dedicated to transforming the world of AI. We are looking for talented individuals who share our mission to democratize machine learning. Positions are available in various domains, including engineering, product management, and community engagement.

- **Why Work With Us?**
  - A stimulating environment that encourages initiative and innovation.
  - Opportunities for professional growth and development.
  - Be part of a community harnessing the power of AI for good.

Explore career openings on our [Jobs page](https://huggingface.co/jobs) and contribute to shaping the future of AI.

---

## Get in Touch

Drive innovation with us at Hugging Face! 

- **Website**: [Hugging Face](https://huggingface.co)
- **Community Pages**: Join us on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/huggingface), and [Discord](https://discord.com/invite/huggingface).
  
Together, let's make machine learning accessible for everyone, one commit at a time!

In [26]:
create_brochure("deeplearning.AI", "https://www.deeplearning.ai/courses/machine-learning-specialization/")


Found links: {'links': [{'type': 'about page', 'url': 'https://www.deeplearning.ai/about/'}, {'type': 'careers page', 'url': 'https://www.deeplearning.ai/careers/'}]}


```markdown
# DeepLearning.AI Brochure

Welcome to **DeepLearning.AI**, a leader in artificial intelligence education and innovation.

## Company Overview
Founded by the renowned AI pioneer **Andrew Ng**, co-founder of Google Brain and Coursera, DeepLearning.AI is dedicated to empowering individuals with knowledge and skills in artificial intelligence and machine learning. Our transformative educational offerings, including the **Machine Learning Specialization**, equip learners with the foundational and advanced concepts necessary to thrive in the AI landscape.

### Our Courses
- **Machine Learning Specialization**: An updated foundational program designed specifically for beginners. This course comprises three comprehensive modules that balance intuition, hands-on coding, and mathematical theory to create an effective learning experience.
- **Deep Learning Specialization**: Take the next step in your AI journey with our advanced deep learning courses.

## Company Culture
At DeepLearning.AI, we believe in fostering a supportive and inclusive environment where learners from all backgrounds can thrive. Our culture emphasizes **innovation**, **collaboration**, and **continuous learning**, empowering our community to advance the field of AI.

### Community Engagement
- **Forums**: Engage with fellow learners and industry experts.
- **Events**: Participate in a range of workshops and networking events to expand your knowledge and connections within the AI community.
- **Ambassador Program**: Join our ambassador program to gain additional learning opportunities and showcase your skills.

## Our Customers
DeepLearning.AI has reached **over 4.8 million learners** from diverse fields, including:
- Data Engineering
- Computational Science
- Behavioral Science
- Financial Services

### Success Stories
Our learners have found great success by applying knowledge gained from our courses:
- **Chirag Godawat** landed a job at JP Morgan Chase.
- **Aakash Saroop** authored a research paper and secured an internship at Morgan Stanley.
- **Nicholas Muchinguri** automated investment processes after gaining insights from our Machine Learning course.

## Careers at DeepLearning.AI
We are always looking for passionate individuals to join our team. If you love AI and education and are eager to make a difference, consider a career with us! Explore available positions on our **Careers** page to see how you can contribute to our mission.

### Join Us Today!
Whether you are looking to **break into AI**, enhance your skills, or explore a career with us, DeepLearning.AI is your partner in innovation and growth.

[Start Learning Now](#) | [Explore Careers](#) 
```


In [27]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [28]:
stream_brochure("HuggingFace", "https://huggingface.co")


Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Company Overview
**Hugging Face** is an innovative AI community dedicated to building the future of machine learning (ML). It serves as a collaborative platform for users to create, discover, and share over 1 million models, 250,000 datasets, and numerous applications. Hugging Face fosters a vibrant community where researchers and developers can work together to advance the field of artificial intelligence.

## Our Mission
We aim to accelerate the adoption of machine learning by providing tools and resources that make it easier for everyone to engage with and deploy AI.

## Key Offerings
- **Models & Datasets:** Access to a vast library of pre-trained models and datasets, supporting a wide range of modalities such as text, images, video, and audio.
- **Spaces:** A platform for users to build and deploy applications quickly and effortlessly.
- **Enterprise Solutions:** Advanced features tailored for organizations, including enterprise-grade security, dedicated support, and scalable compute options starting at just $20/user/month.
- **Open Source:** Hugging Face is committed to open-source contributions, providing tools like Transformers, Diffusers, and Tokenizers for the community to build upon.

## Customers & Collaborations
Hugging Face is trusted by over 50,000 organizations, including industry giants such as:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Grammarly**

Darwin among them, these organizations utilize Hugging Face’s offerings to enhance their AI capabilities and drive innovation.

## Company Culture
At Hugging Face, we pride ourselves on a collaborative and inclusive work environment. We embrace transparency, encourage knowledge sharing, and foster creativity. Our culture promotes continuous learning, where team members are encouraged to explore their interests in AI and ML. 

## Career Opportunities
Hugging Face is actively looking for talented individuals to join our team. With a wide range of career prospects available, we invite you to explore the current openings in various domains within the AI field. 

If you're passionate about machine learning and want to make an impact, consider joining us in our mission to democratize AI!

## Get In Touch!
To learn more about Hugging Face, explore our resources, or join the community, please visit our [website](https://huggingface.co) or follow us on our social platforms.

---

Join us in building the future of AI together!

In [29]:
stream_brochure("Deeplearning.AI", "https://www.deeplearning.ai/courses/machine-learning-specialization/")


Found links: {'links': [{'type': 'about page', 'url': 'https://www.deeplearning.ai/about/'}, {'type': 'careers page', 'url': 'https://www.deeplearning.ai/careers/'}]}


# Deeplearning.AI Brochure

---

## Welcome to Deeplearning.AI

### Your Gateway to AI Education

At Deeplearning.AI, we believe that understanding AI and machine learning should be accessible to everyone. With our renowned courses designed by AI pioneer Andrew Ng, we help learners break into the world of AI.

---

## Courses Offered

### Machine Learning Specialization

- **New Course:** Orchestrating Workflows for GenAI Applications
- **Enroll in:** Machine Learning Specialization, a foundational program perfect for beginners.
- **Course Rating:** 4.9 out of 5 from 170K learners.
- **Total Learners:** 4.8 million people have embarked on their AI journey with us.

#### Course Highlights:
- **No Prior Experience Required:** The course is designed for complete beginners and does not necessitate a rigorous coding background.
- **Visual Learning Approach:** Each lesson begins with intuitive visuals followed by practical coding examples.
- **Updated Content:** The curriculum has been rebuilt to use Python and incorporates best practices from the last decade in machine learning.

---

## What Our Learners Say

Hear from our learners who have transformed their careers through our courses:

- **Luo Yuzheng** (Monetary Authority of Singapore): "I adopted methods to understand my customers better, breaking away from the one-size-fits-all approach."
- **Chirag Godawat** (Vista): "I gained confidence and landed a job at JP Morgan Chase after completing the course."
- **Hsin-Wen Chang** (Zealogics): "Andrew Ng's concepts became a guiding light that helped me explain outcomes to my colleagues."

---

## Community Engagement

Join our vibrant community where learners can engage through forums, newsletters, and events:

- **The Batch:** Our AI newsletter featuring updates and insights from the AI world.
- **Ambassadors:** Connect with community members who advocate for AI education.

---

## Company Culture

At Deeplearning.AI, we champion a culture of learning, curiosity, and inclusivity. Our team is committed to making AI education innovative and approachable, focusing on:

- **Diversity:** We value diverse perspectives and strive for an inclusive environment.
- **Collaboration:** Teamwork drives our initiatives as we work together to expand the horizons of AI.

---

## Careers

Are you passionate about AI and education? Join our team and help shape the future of machine learning learning:

- **Open Positions:** Check out our careers page for current job openings.
- **Career Growth:** We provide ample opportunities for professional development and career advancement.

---

## Contact Us

Ready to level up your AI skills or learn more about joining our team? Reach out to us:

- **Website:** [Deeplearning.AI](https://www.deeplearning.ai)
- **Email:** info@deeplearning.ai
- **Follow Us:** Stay connected on our social platforms for updates.

---

### Start Your AI Journey Today!

**Enroll Now and Break Into AI!**

--- 

Join us at Deeplearning.AI to empower your future with AI capabilities!